This notebook imports data that were scraped from BoxOfficeMojo in previous notebooks. The data include:

 - Movie Title, Studio, Opening, Budget, Earliest Release Date, MPAA Rating, Runtime, Genres, Domestic Gross, International Gross

Workflow:
Package into a pandas DataFrame, 





In [1]:
import requests
import pickle
import pandas as pd
import numpy as np
import matplotlib as plt

### Read in the data, create a DataFrame

In [2]:
# Read in the list of dictionaries with movie data
with open('movies_data_raw.pickle','rb') as read_file:
    movies_raw = pickle.load(read_file)

In [3]:
# convert to a data frame
movies_df = pd.DataFrame(movies_raw)
movies_df

,Title,Studio,Opening,Budget,Release,Rating,Runtime,Genre,Domestic,International
0,Toy Story 4,Walt Disney Studios Motion Pictures,"$120,908,065","$200,000,000","June 20, 2019",G,1 hr 40 min,"[Adventure, Animation, Comedy, Family, Fantasy]","$434,038,008","$639,356,585"
1,The Lion King,Walt Disney Studios Motion Pictures,"$1,586,753","$45,000,000","June 15, 1994",G,1 hr 28 min,"[Adventure, Animation, Drama, Family, Musical]","$422,783,777","$545,728,028"
2,Toy Story 3,Walt Disney Studios Motion Pictures,"$110,307,189","$200,000,000","June 16, 2010",na,1 hr 43 min,"[Adventure, Animation, Comedy, Family, Fantasy]","$415,004,880","$651,964,823"
3,Finding Nemo,Walt Disney Studios Motion Pictures,"$70,251,710","$94,000,000","May 30, 2003",na,1 hr 40 min,"[Adventure, Animation, Comedy, Family]","$380,843,261","$559,506,933"
4,"Monsters, Inc.",Walt Disney Studios Motion Pictures,"$62,577,067","$115,000,000","November 2, 2001",G,1 hr 32 min,"[Adventure, Animation, Comedy, Family, Fantasy]","$289,916,256","$289,064,814"
...,...,...,...,...,...,...,...,...,...,...
3358,Three Days of the Condor,Paramount Pictures,na,na,"September 24, 1975",na,1 hr 57 min,"[Mystery, Thriller]","$27,476,252","$27,476,252"
3359,Friday,New Line Cinema,"$6,589,341","$3,500,000","April 28, 1995",R,1 hr 31 min,"[Comedy, Drama]","$27,467,564","$748,354"
3360,The Brothers,Screen Gems,"$10,302,846","$6,000,000","March 23, 2001",R,1 hr 46 min,"[Comedy, Drama]","$27,457,409","$500,782"
3361,Midsommar,A24,"$6,560,030","$9,000,000","July 3, 2019",R,2 hr 28 min,"[Drama, Horror, Mystery, Thriller]","$27,426,361","$20,476,738"


In [4]:
# Change 'na' to np.nan
movies_df.replace('na',np.nan, inplace=True)

In [5]:
# Get a report of 'na' fields by feature
movies_df.apply(lambda x: sum(x.isna()), axis=0)

Title               0
Studio              7
Opening           251
Budget           1386
Release             2
Rating            859
Runtime             0
Genre               0
Domestic            0
International       0
dtype: int64

### Impute ratings

In [6]:
# Movies are sorted by MPAA rating
# That let's us impute missing values in the rating column
for index, row in movies_df.iterrows():
    if row['Rating'] in ['G','PG','PG-13','R']:
        current_rating = row['Rating']
    else:
        row['Rating'] = current_rating

In [7]:
movies_df

,Title,Studio,Opening,Budget,Release,Rating,Runtime,Genre,Domestic,International
0,Toy Story 4,Walt Disney Studios Motion Pictures,"$120,908,065","$200,000,000","June 20, 2019",G,1 hr 40 min,"[Adventure, Animation, Comedy, Family, Fantasy]","$434,038,008","$639,356,585"
1,The Lion King,Walt Disney Studios Motion Pictures,"$1,586,753","$45,000,000","June 15, 1994",G,1 hr 28 min,"[Adventure, Animation, Drama, Family, Musical]","$422,783,777","$545,728,028"
2,Toy Story 3,Walt Disney Studios Motion Pictures,"$110,307,189","$200,000,000","June 16, 2010",G,1 hr 43 min,"[Adventure, Animation, Comedy, Family, Fantasy]","$415,004,880","$651,964,823"
3,Finding Nemo,Walt Disney Studios Motion Pictures,"$70,251,710","$94,000,000","May 30, 2003",G,1 hr 40 min,"[Adventure, Animation, Comedy, Family]","$380,843,261","$559,506,933"
4,"Monsters, Inc.",Walt Disney Studios Motion Pictures,"$62,577,067","$115,000,000","November 2, 2001",G,1 hr 32 min,"[Adventure, Animation, Comedy, Family, Fantasy]","$289,916,256","$289,064,814"
...,...,...,...,...,...,...,...,...,...,...
3358,Three Days of the Condor,Paramount Pictures,NaN,NaN,"September 24, 1975",R,1 hr 57 min,"[Mystery, Thriller]","$27,476,252","$27,476,252"
3359,Friday,New Line Cinema,"$6,589,341","$3,500,000","April 28, 1995",R,1 hr 31 min,"[Comedy, Drama]","$27,467,564","$748,354"
3360,The Brothers,Screen Gems,"$10,302,846","$6,000,000","March 23, 2001",R,1 hr 46 min,"[Comedy, Drama]","$27,457,409","$500,782"
3361,Midsommar,A24,"$6,560,030","$9,000,000","July 3, 2019",R,2 hr 28 min,"[Drama, Horror, Mystery, Thriller]","$27,426,361","$20,476,738"


### Drop NA rows

In [8]:
# Remove NAs from Budget and Opening
movies_df.dropna(subset=['Budget','Opening'],axis=0,inplace=True)

In [9]:
# Check if any NAs remaining
movies_df.apply(lambda x: sum(x.isna()), axis=0)

Title            0
Studio           0
Opening          0
Budget           0
Release          0
Rating           0
Runtime          0
Genre            0
Domestic         0
International    0
dtype: int64

In [10]:
movies_df.shape

(1935, 10)

### Convert Release Date to DateTime object

In [11]:
# Convert Release Date, drop the original
movies_df['ReleaseDate'] = pd.to_datetime(movies_df['Release'])
movies_df.drop(columns=['Release'], inplace=True, axis=1)

### Convert Runtime string to minutes integer

In [12]:
# Some runtimes list hours and minutes, some only list hours
# Use runtime_to_minutes to convert runtime string to an integer of total minutes

def runtime_to_minutes(row):
    """
    Split runtime string into hours and minutes (or just hours),
    convert values to integers, and return runtime as an integer of total minutes
    """
    time_list = row['Runtime'].split(' ')
    if len(time_list) == 2:
        tot_mins = int(time_list[0])*60
    else:
        tot_mins = int(time_list[0])*60 + int(time_list[2])
    return tot_mins

movies_df['Runtime'] = movies_df.apply(runtime_to_minutes, axis = 1)

### Convert currency to integer values

In [25]:
# Convert currency strings to integers so they can be used as a continuous features
# Also set international earnings to 0 if scraping algorithm incorrectly assigned international==domestic
def currency_to_int(row):
    """
    Convert currency string in columns 'Opening', 'Budget', 'Domestic' and 'International' to integers.
    In cases where scraping algorithm set undefined international earnings equal to domestic, 
    replace international earnings with 0.
    """
    opening = int(row['Opening'].replace('$','').replace(',',''))
    budget = int(row['Budget'].replace('$','').replace(',',''))
    domestic = int(row['Domestic'].replace('$','').replace(',',''))
    international = int(row['International'].replace('$','').replace(',',''))
    if domestic == international:
        return opening, budget, domestic, 0
    else:
        return opening, budget, domestic, international

movies_df[['Opening','Budget','Domestic','International']] = list(movies_df.apply(currency_to_int, axis = 1))

In [26]:
movies_df

,Title,Studio,Opening,Budget,Rating,Runtime,Genre,Domestic,International,ReleaseDate
0,Toy Story 4,Walt Disney Studios Motion Pictures,120908065,200000000,G,100,"[Adventure, Animation, Comedy, Family, Fantasy]",434038008,639356585,2019-06-20
1,The Lion King,Walt Disney Studios Motion Pictures,1586753,45000000,G,88,"[Adventure, Animation, Drama, Family, Musical]",422783777,545728028,1994-06-15
2,Toy Story 3,Walt Disney Studios Motion Pictures,110307189,200000000,G,103,"[Adventure, Animation, Comedy, Family, Fantasy]",415004880,651964823,2010-06-16
3,Finding Nemo,Walt Disney Studios Motion Pictures,70251710,94000000,G,100,"[Adventure, Animation, Comedy, Family]",380843261,559506933,2003-05-30
4,"Monsters, Inc.",Walt Disney Studios Motion Pictures,62577067,115000000,G,92,"[Adventure, Animation, Comedy, Family, Fantasy]",289916256,289064814,2001-11-02
...,...,...,...,...,...,...,...,...,...,...
3353,Saw VI,Lionsgate,14118444,11000000,R,90,"[Horror, Mystery]",27693292,40540862,2009-10-22
3359,Friday,New Line Cinema,6589341,3500000,R,91,"[Comedy, Drama]",27467564,748354,1995-04-28
3360,The Brothers,Screen Gems,10302846,6000000,R,106,"[Comedy, Drama]",27457409,500782,2001-03-23
3361,Midsommar,A24,6560030,9000000,R,148,"[Drama, Horror, Mystery, Thriller]",27426361,20476738,2019-07-03


In [ ]:
# Parse genres out into separate columns
genre_df = pd.get_dummies(movies_df['Genre'].apply(pd.Series).stack()).sum(level=0)
movies_df = pd.concat([movies_df, genre_df], axis=1)

# Drop Genre column
df_combined.drop(columns=['DATE','TIME'], inplace=True, axis=1)
movies_df

In [ ]:
# Set Studio to dummy variable

# First get a list of studios that account for 90% of movies in the df
studio_counts = movies_df['Studio'].value_counts().reset_index()
top_studios = []
count = 0
i=0
while count < 0.9*len(movies_df):
    top_studios.append(studio_counts['index'].iloc[i])
    count+= studio_counts['Studio'].iloc[i]
    i+=1

    







In [ ]:
# Convert Runtime to a datetime object:
"""day1['time'] = pd.to_datetime(day1['time'], format='%H:%M').dt.time"""